In [2]:
import torch
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, random_split

In [3]:
import torch
import torchvision.models as models
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, random_split
from torch import nn, optim
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
# Load a pre-trained ResNet model
model = models.resnet50(pretrained=True)

# Freeze all the parameters in the model
for param in model.parameters():
    param.requires_grad = False

# Replace the final fully connected layer
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 1)  # For binary classification

# Check if a GPU is available and move the model to GPU if it is

model = model.to(device)

C:\Users\user\AppData\Roaming\Python\Python310\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\user\AppData\Roaming\Python\Python310\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [5]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize the image to 224x224 for ResNet
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Normalization for ResNet
])


In [8]:

# Load your dataset
dataset = datasets.ImageFolder("./", transform=transform)

# Splitting dataset into train and validation
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [9]:
dataset

Dataset ImageFolder
    Number of datapoints: 336
    Root location: ./
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )

In [10]:
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

In [11]:
num_epochs = 100  # Define the number of epochs

for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0
    
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()  # Zero the parameter gradients
        
        outputs = model(inputs)  # Forward pass
        loss = criterion(outputs, labels.unsqueeze(1).type_as(outputs))  # Calculate loss
        loss.backward()  # Backward pass
        optimizer.step()  # Optimize
        
        running_loss += loss.item()
    
    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}")

Epoch 1, Loss: 0.6320125129487779
Epoch 2, Loss: 0.5223452515072293
Epoch 3, Loss: 0.4587623675664266
Epoch 4, Loss: 0.4418993592262268
Epoch 5, Loss: 0.4408339493804508
Epoch 6, Loss: 0.37118057078785366
Epoch 7, Loss: 0.3573167986339993
Epoch 8, Loss: 0.3436223566532135
Epoch 9, Loss: 0.3381568259663052
Epoch 10, Loss: 0.34341032968627083
Epoch 11, Loss: 0.37047525743643445
Epoch 12, Loss: 0.2899395128091176
Epoch 13, Loss: 0.3156075229247411
Epoch 14, Loss: 0.29181301759348977
Epoch 15, Loss: 0.26763605574766797
Epoch 16, Loss: 0.2824435813559426
Epoch 17, Loss: 0.26815664437082076
Epoch 18, Loss: 0.265707415011194
Epoch 19, Loss: 0.2518082824018266
Epoch 20, Loss: 0.23313684430387285
Epoch 21, Loss: 0.2446866234143575
Epoch 22, Loss: 0.24073375099235111
Epoch 23, Loss: 0.21522613035307991
Epoch 24, Loss: 0.24697492354445988
Epoch 25, Loss: 0.2230644706222746
Epoch 26, Loss: 0.2315814710325665
Epoch 27, Loss: 0.21983419524298775
Epoch 28, Loss: 0.19113872283034855
Epoch 29, Loss: 0.

bad


In [ ]:
import torch
from torchvision import transforms
 # For finding image files

# Define image path (replace with your actual folder paths)
healthy_path = "./HEALTHY/"
cancer_path = "./CANCER/"

# Define transformations (same as previous example)
transform = transforms.Compose([
    transforms.Resize(128),
    transforms.ToTensor(),
])

# Function to load images from a folder
def load_images(folder_path, label):
  images = []
  for image_path in glob(f"{folder_path}*.png"):  # Modify extension if needed (.png, etc.)
    image = Image.open(image_path)
    image_tensor = transform(image).unsqueeze(0)  # Add batch dimension
    images.append((image_tensor, label))
   
  return images

# Load healthy and cancer images
healthy_images = load_images(healthy_path, 0)  # Assuming 0 represents healthy
cancer_images = load_images(cancer_path, 1)  # Assuming 1 represents cancer

# Combine images into a single list (optional)
all_images = healthy_images + cancer_images

In [47]:
len()

1

In [34]:
len(healthy_images)

133

In [35]:
len(cancer_images)

203

In [ ]:
model = models.vgg16(pretrained=True)
for param in model.parameters():
    param.requires_grad = False  # Freeze weights (optional)
model.to('cpu')
model.eval()


RuntimeError: Given groups=1, weight of size [64, 3, 3, 3], expected input[1, 1, 128, 128] to have 3 channels, but got 1 channels instead